In [1]:
import os

os.environ["CV_IO_MAX_IMAGE_PIXELS"] = f"{2**99}"
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = f"{2**63}"

import numpy as np
import matplotlib.pyplot as plt
import json
import cv2
import tiffslide

In [2]:
def get_binary_mask(wsi_path, annotations_path):
    with open(annotations_path) as f:
        annotations = json.load(f)
    # Load the WSI file
    slide = tiffslide.open_slide(wsi_path)
    dimensions = slide.dimensions
    width, height = dimensions
    
    binary_mask = np.zeros((height, width), dtype=np.uint8)
    if "annotation" in annotations:
        elements = annotations['annotation']['elements']
    else:
        elements = annotations[0]['elements']
    for element in elements:
        if element['type'] == 'polyline':
            points = element['points']
            # print(len(points[0]))
            if element['closed'] and len(points) > 0 and len(points[0]) == 3:
                points = np.array(points, dtype=np.int32)
                # Ensure the points are in the correct shape (N, 1, 2)
                points = np.array([[p[0], p[1]] for p in points], dtype=np.int32)
                points = points.reshape((-1, 1, 2))
                contours = [points]
                cv2.drawContours(binary_mask, contours, -1, color=255, thickness=cv2.FILLED)  # Draw the filled contour

    slide.close()
    # print('Done!')
    return binary_mask
    

In [ ]:


WSI_images_dir = "/blue/pinaki.sarder/manojkumargalla/PostProcess/data/model2/set2/wsis_1"
tbm_annotations_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/model2/set2/TBM_Annotations/'
tubules_annotations_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/newmodel/Tubules_annotations_set1/'
filtered_wsi_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/newmodel/Filtered_WSI_images_set1/'

WSI_images = os.listdir(WSI_images_dir)
print(WSI_images)


for wsi in WSI_images:
    wsi_path = os.path.join(WSI_images_dir, wsi)
    wsi_name = str(wsi.split('.')[0])
    print(f'Processing image - {wsi_name}')
    tbm_annotations_path = os.path.join(tbm_annotations_dir, f'{wsi_name}_tbm_jul18.json')
    tubules_annotations_path = os.path.join(tubules_annotations_dir, f'{wsi_name}.json')
    print('Generating binary masks')
    binary_mask_of_tubules = get_binary_mask(wsi_path, tubules_annotations_path)
    binary_mask_of_tbms = get_binary_mask(wsi_path, tbm_annotations_path)
    print('performing distancec transform')
    distance_transform = cv2.distanceTransform(binary_mask_of_tubules, cv2.DIST_L2, 5)
    inverse_binary_mask = 255 - binary_mask_of_tubules
    inverse_distance_transform = cv2.distanceTransform(inverse_binary_mask, cv2.DIST_L2, 5)
    distance_map = distance_transform - inverse_distance_transform
    threshold = 25
    mask = np.abs(distance_map) < threshold
    # Convert boolean mask to binary (0 and 255)
    threshInv = np.where(mask, 255, 0).astype(np.uint8)
    product = threshInv * binary_mask_of_tbms
    print(f'Saving filtered image - {wsi_name}')
    filtered_image_path = os.path.join(filtered_wsi_dir, f'{wsi_name}.png')
    cv2.imwrite(filtered_image_path, 255*product)
    print(f'Succesfully filtered image - {wsi_image}')
        
    

In [3]:

from concurrent.futures import ThreadPoolExecutor

def save_image_masks(wsi, WSI_images_dir, tbm_annotations_dir, tubules_annotations_dir, binaries_dir):
    wsi_path = os.path.join(WSI_images_dir, wsi)
    wsi_name = str(wsi.split('.')[0])
    print(f'Processing image - {wsi_name}')
    tbm_annotations_path = os.path.join(tbm_annotations_dir, f'{wsi_name}_tbm_test.json')
    # tubules_annotations_path = os.path.join(tubules_annotations_dir, f'{wsi_name}_tubules.json')
    tubules_annotations_path = os.path.join(tubules_annotations_dir, f'{wsi_name}.json')
    print('Generating binary masks')
    binary_mask_of_tubules = get_binary_mask(wsi_path, tubules_annotations_path)
    binary_mask_of_tbms = get_binary_mask(wsi_path, tbm_annotations_path)
    binary_path = os.path.join(binaries_dir, f'{wsi_name}')
    if not os.path.exists(binary_path):
        os.mkdir(binary_path)
    print('Saving binary mask numpy arrays')
    np.save(f'{binary_path}/binary_mask_of_tbms', binary_mask_of_tbms)
    np.save(f'{binary_path}/binary_mask_of_tubules', binary_mask_of_tubules)


def main():
    WSI_images_dir = "/blue/pinaki.sarder/manojkumargalla/PostProcess/data/model2/batch1/3/wsis_3"
    tbm_annotations_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/model2/batch1/3/TBM_Annotations_3/'
    tubules_annotations_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/model2/batch1/1/Tubules_Annotations'
    binaries_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/model2/batch1/3/binaries/'

    WSI_images = os.listdir(WSI_images_dir)
    print(WSI_images)

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(save_image_masks, wsi, WSI_images_dir, tbm_annotations_dir, tubules_annotations_dir, binaries_dir) for wsi in WSI_images]
        for future in futures:
            future.result()  # This will raise exceptions if any

main()


['S-2010-012854_PAS_2of2.svs', 'S-2010-004324_PAS_2of2.svs', 'S-2010-004278_PAS_1of2.svs', 'S-2006-005000_PAS_1of2.svs', 'S-2006-004953_PAS_1of2.svs', 'S-2010-012855_PAS_1of2.svs', 'S-2001-005357_PAS_1of2.svs', 'S-2006-005047_PAS_1of2.svs', 'S-2006-004811_PAS_1of2.svs', 'S-2006-002095_PAS_2of3.svs', 'S-2006-005094_PAS_1of2.svs', 'S-2006-002045_PAS_1of2.svs', 'S-2010-004277_PAS_2of2.svs', 'S-2006-002084_PAS_1of3.svs', 'S-2006-002044_PAS_2of2.svs', 'S-1909-007149_PAS_1of2.svs', 'S-2010-004184_PAS_1of2.svs', 'S-1905-017738_PAS_1of2.svs', 'S-2010-004230_PAS_2of2.svs']
Processing image - S-2010-012854_PAS_2of2
Generating binary masks
Processing image - S-2010-004324_PAS_2of2
Generating binary masks
Processing image - S-2010-004278_PAS_1of2
Generating binary masks
Processing image - S-2006-005000_PAS_1of2
Generating binary masks
Saving binary mask numpy arrays
Saving binary mask numpy arrays
Processing image - S-2006-004953_PAS_1of2
Generating binary masks
Saving binary mask numpy arrays
Pro

In [ ]:
from concurrent.futures import ThreadPoolExecutor
def process_image(binaries_dir, wsi, filtered_wsi_dir):
    wsi_name = str(wsi.split('.')[0])
    if os.path.exists(os.path.join(filtered_wsi_dir, f'{wsi_name}.png')):
        path_ = os.path.join(filtered_wsi_dir, f'{wsi_name}.png')
        print(f'{path_} already exists')
        return
    binary_path = os.path.join(binaries_dir, wsi_name)
    binary_mask_of_tubules = np.load(f'{binary_path}/binary_mask_of_tubules.npy')
    binary_mask_of_tbms = np.load(f'{binary_path}/binary_mask_of_tbms.npy')
    print('performing distance transform')
    distance_transform = cv2.distanceTransform(binary_mask_of_tubules, cv2.DIST_L2, 5)
    inverse_binary_mask = 255 - binary_mask_of_tubules
    inverse_distance_transform = cv2.distanceTransform(inverse_binary_mask, cv2.DIST_L2, 5)
    distance_map = distance_transform - inverse_distance_transform
    threshold = 10
    mask = np.abs(distance_map) < threshold
    # Convert boolean mask to binary (0 and 255)
    threshInv = np.where(mask, 255, 0).astype(np.uint8)
    product = threshInv * binary_mask_of_tbms
    print(f'Saving filtered image - {wsi_name}')
    filtered_image_path = os.path.join(filtered_wsi_dir, f'{wsi_name}.png')
    cv2.imwrite(filtered_image_path, 255 * product)
    print(f'Successfully filtered image - {wsi_name}')

def main():
    WSI_images_dir = "/blue/pinaki.sarder/manojkumargalla/PostProcess/data/new_model/pilot_wsis_set2/"
    filtered_wsi_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/new_model/Filtered_WSI_binary_images_set2_newmodel'
    binaries_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/new_model/binaries_newmodel_set2/'
    
    WSI_images = os.listdir(WSI_images_dir)
    print(WSI_images)

    # with ThreadPoolExecutor(max_workers=8) as executor:
    #     futures = [executor.submit(process_image, binaries_dir, wsi, filtered_wsi_dir) for wsi in WSI_images]
    #     for future in futures:
    #         future.result()  
    
    for wsi in WSI_images:
      process_image(binaries_dir, wsi, filtered_wsi_dir)  
            
main()
    

In [ ]:
import gc

def process_image(binaries_dir, wsi, filtered_wsi_dir):
    wsi_name = str(wsi.split('.')[0])
    if os.path.exists(os.path.join(filtered_wsi_dir, f'{wsi_name}.png')):
        path_ = os.path.join(filtered_wsi_dir, f'{wsi_name}.png')
        print(f'{path_} already exists')
        return
    binary_path = os.path.join(binaries_dir, wsi_name)
    binary_mask_of_tubules = np.load(f'{binary_path}/binary_mask_of_tubules.npy')
    binary_mask_of_tbms = np.load(f'{binary_path}/binary_mask_of_tbms.npy')
    print(f'performing distance transform - {wsi_name}')
    distance_transform = cv2.distanceTransform(binary_mask_of_tubules, cv2.DIST_L2, 5)
    inverse_binary_mask = 255 - binary_mask_of_tubules
    inverse_distance_transform = cv2.distanceTransform(inverse_binary_mask, cv2.DIST_L2, 5)
    distance_map = distance_transform - inverse_distance_transform
    
    del binary_mask_of_tubules  # Delete array after usage
    del inverse_binary_mask     # Delete array after usage
    del distance_transform      # Delete array after usage
    del inverse_distance_transform  # Delete array after usage
    gc.collect()
    
    threshold = 10
    mask = np.abs(distance_map) < threshold
    # Convert boolean mask to binary (0 and 255)
    threshInv = np.where(mask, 255, 0).astype(np.uint8)
    product = threshInv * binary_mask_of_tbms
    
    del distance_map  # Delete array after usage
    del mask          # Delete array after usage
    del threshInv 
    gc.collect()
    
    print(f'Saving filtered image - {wsi_name}')
    filtered_image_path = os.path.join(filtered_wsi_dir, f'{wsi_name}.png')
    cv2.imwrite(filtered_image_path, 255 * product)
    
    del binary_mask_of_tbms  # Delete array after usage
    del product  
    
    gc.collect()
    print(f'Successfully filtered image - {wsi_name}')

def main():
    WSI_images_dir = "/blue/pinaki.sarder/manojkumargalla/PostProcess/data/model2/batch1/3/wsis_3"
    filtered_wsi_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/model2/batch1/3/filtered_wsis_3/'
    binaries_dir = '/blue/pinaki.sarder/manojkumargalla/PostProcess/data/model2/batch1/3/binaries/'
    
    WSI_images = os.listdir(WSI_images_dir)
    print(WSI_images)

    # with ThreadPoolExecutor(max_workers=8) as executor:
    #     futures = [executor.submit(process_image, binaries_dir, wsi, filtered_wsi_dir) for wsi in WSI_images]
    #     for future in futures:
    #         future.result()  
    
    for wsi in WSI_images:
      process_image(binaries_dir, wsi, filtered_wsi_dir)  
            
main()

['S-2010-012854_PAS_2of2.svs', 'S-2010-004324_PAS_2of2.svs', 'S-2010-004278_PAS_1of2.svs', 'S-2006-005000_PAS_1of2.svs', 'S-2006-004953_PAS_1of2.svs', 'S-2010-012855_PAS_1of2.svs', 'S-2001-005357_PAS_1of2.svs', 'S-2006-005047_PAS_1of2.svs', 'S-2006-004811_PAS_1of2.svs', 'S-2006-002095_PAS_2of3.svs', 'S-2006-005094_PAS_1of2.svs', 'S-2006-002045_PAS_1of2.svs', 'S-2010-004277_PAS_2of2.svs', 'S-2006-002084_PAS_1of3.svs', 'S-2006-002044_PAS_2of2.svs', 'S-1909-007149_PAS_1of2.svs', 'S-2010-004184_PAS_1of2.svs', 'S-1905-017738_PAS_1of2.svs', 'S-2010-004230_PAS_2of2.svs']
performing distance transform - S-2010-012854_PAS_2of2
Saving filtered image - S-2010-012854_PAS_2of2
Successfully filtered image - S-2010-012854_PAS_2of2
performing distance transform - S-2010-004324_PAS_2of2
Saving filtered image - S-2010-004324_PAS_2of2
Successfully filtered image - S-2010-004324_PAS_2of2
performing distance transform - S-2010-004278_PAS_1of2
Saving filtered image - S-2010-004278_PAS_1of2
Successfully filt

In [ ]:
binary_mask_of_tubules = np.load('/blue/pinaki.sarder/manojkumargalla/PostProcess/data/binaries/S-2103-004857_PAS_2of2/binary_mask_of_tubules.npy')
binary_mask_of_tbms = np.load('/blue/pinaki.sarder/manojkumargalla/PostProcess/data/binaries/S-2103-004857_PAS_2of2/binary_mask_of_tbms.npy')
# cv2.imwrite('/blue/pinaki.sarder/manojkumargalla/test_postprocess/S-2103-004857_PAS_2of2_tbms_unfiltered.png', binary_mask_of_tbms)
# cv2.imwrite('/blue/pinaki.sarder/manojkumargalla/test_postprocess/S-2103-004857_PAS_2of2_tubules.png', binary_mask_of_tubules)
# print('saved Binary masks of tubules and unfiltered TBMs')

In [ ]:
distance_transform = cv2.distanceTransform(binary_mask_of_tubules, cv2.DIST_L2, 5)
inverse_binary_mask = 255 - binary_mask_of_tubules
inverse_distance_transform = cv2.distanceTransform(inverse_binary_mask, cv2.DIST_L2, 5)
distance_map = distance_transform - inverse_distance_transform
threshold = 10
mask = np.abs(distance_map) < threshold
# Convert boolean mask to binary (0 and 255)
threshInv = np.where(mask, 255, 0).astype(np.uint8)
product = threshInv * binary_mask_of_tbms
cv2.imwrite('/blue/pinaki.sarder/manojkumargalla/test_postprocess/S-2103-004857_PAS_2of2_tbms_filtered.png', 255*product)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import glob
import matplotlib.image as mpimg

images_folder = '/home/manojkumargalla/temp_images'
# List of image file paths
image_paths = glob.glob(f'{images_folder}/*.png')
print(image_paths)

# Create a figure with a 3x2 grid of subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Loop through image paths and axes to display each image
for i, (ax, img_path) in enumerate(zip(axes, image_paths)):
    img = mpimg.imread(img_path)
    ax.imshow(img)
    ax.axis('off')  # Hide axes for a cleaner look
    image_name = img_path.split('/')[-1]
    ax.set_title(f'{image_name}')

# Remove any unused subplots
for j in range(len(image_paths), len(axes)):
    fig.delaxes(axes[j])

# Adjust layout and show plot
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

In [ ]:
binary_tbm = np.load('/blue/pinaki.sarder/manojkumargalla/PostProcess/data/binaries/18-162_PAS_4of6/binary_mask_of_tbms.npy')
binary_tubules = np.load('/blue/pinaki.sarder/manojkumargalla/PostProcess/data/binaries/18-162_PAS_4of6/binary_mask_of_tubules.npy')

In [ ]:
import cv2

In [ ]:
cv2.imwrite('binary_tbm_18-162_PAS_4of6.png', binary_tbm)
cv2.imwrite('binary_tubules_18-162_PAS_4of6.png', binary_tubules)